In [1]:
import numpy as np

<img align="center" src="figures/course.png" width="800">

## Neural Networks for Recognition - Assignment 3
    Instructor: Kris                          TAs: Arka, Jinkun, Rawal, Rohan, Sheng-Yu

---
## Theory Questions (45 points)
**Grading**:  
- The theory part consists of 7 questions.
- Please add your answers to the writeup (submitted as pdf to HW3:PDF). Insert images whenever necessary.
- Show all your work to obtain full credit.

# From coding qs, writeups:

#### Q1.1.1 (3 points, write-up)
Why is it not a good idea to initialize a network with all zeros? If you imagine that every layer has weights and biases, what can a zero-initialized network output after training?

Since the weights are multipliers at each layer, if you initialize a network with all zeroes, it may just an output vector of all zeros as well. 


#### Q1.1.3 (2 points, write-up)
Why is it a good practice to initialize the parameters using random numbers? Explain the intuition behind scaling the initializations depending on layer size (see near Fig 6 in [Xavier initialization](http://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf))?

It is good practice to initialize the parameters with random numbers because if you initialize them all with the same constant, the network is not incentivized to distinguish between different weights in training and backprop. 

In Xavier initialization, we also scale the random initializations depending on the layer size. Otherwise, in the standard initialization the variance of the back-propagated
gradient is dependent on the layer, and decreasing. This causes the back-propagated gradients' effect to diminish the further up in the network we get.

Instead, we want to maintain the activation variances and back-propagated gradients variance as one moves up or down the network, and Xavier initialization achieves that via scaling by the layer size.


# Theory qs, writeups:

## Q1 (3 points)

The softmax function can be defined as, $softmax(x_i)= \frac{1}{S} s_i$ where $s_i = e^{x_i}$ , $S=\sum_i s_i$. Using this definition, please answer Q1.1, Q1.2 and Q1.3 below.

#### Q1.1 (1 point)
Let $x \in \mathbb{R}^d$, what are the properties of $softmax(x)$, specifically, what is the range of each element in $softmax(x)$? What is the sum of all elements in $softmax(x)$?

The range of each element in  $softmax(x)$ = [0,1].

The sum of all elements in  $softmax(x)$ = 1.

#### Q1.2 (1 point)
”Softmax takes an arbitrary real valued vector $x$ and turns it into a ___”. **Please fill in the blank using an appropriate word/phrase**.

probability distribution

#### Q1.3 (1 point)
Let $x \in \mathbb{R}^d$, assume $v = softmax(softmax(... softmax(x)))$ where the softmax function is applied to $x$ recursively $N$ times. What is the value of $v$ as a function of $d$ $\forall x \in \mathbb{R}^d$, in the limit $N \rightarrow \infty$?



If we apply softmax to x recursively, each successive softmax will smooth out the input probability distribution a little more. 

Thus, in the limit $N \rightarrow \infty$, the result will converge to a uniform probability distribution. So $ v=softmax^N(x)$ will approach $ones(n)/n$.

In [2]:
import numpy as np

def softmax(X: np.ndarray):
    """
    A softmax function.
    
    [input]
    * X -- input data [N x D]
    
    [output]
    * res -- values after softmax
    """
    res = None
    
    ## compute res using X
    # YOUR CODE HERE
    # shifted_X  = X - np.max(X, axis=1)

    # # the inputs to each of these is a single x_vector, Nx1
    # softmax_func = np.vectorize(lambda x: np.exp(x)/np.sum(np.exp(x)))
    # # softmax_denom_func =  np.vectorize(lambda x: np.sum(np.exp(x)))

    # res = np.apply_along_axis(softmax_func, 1, shifted_X)

    max_X = np.max(X, axis=1).reshape((-1,1))
    e_X = np.exp(X - max_X)
    softmax_denom = np.sum(e_X, axis=1, keepdims=True)
    res = e_X / softmax_denom

    return res


In [4]:
x = np.array([[0.1, 0.1, 0.2, 0.4, 0.2]])
x.shape

(1, 5)

In [5]:
softmax(x)

array([[0.17984962, 0.17984962, 0.19876458, 0.2427716 , 0.19876458]])

In [6]:
softmax(softmax(x))

array([[0.19595803, 0.19595803, 0.19969984, 0.20868427, 0.19969984]])

In [8]:
softmax(softmax(softmax(softmax(softmax(softmax(softmax(softmax(softmax(softmax(softmax(softmax(softmax(softmax(softmax(softmax(x))))))))))))))))

array([[0.2, 0.2, 0.2, 0.2, 0.2]])

### Q2 (4 points)
Prove that softmax is invariant to translation, that is 
$$softmax(x) = softmax(x + c) \qquad \forall c \in \mathbb{R}$$
Again, softmax is defined as, for each index $i$ in a vector $x$.
$$softmax(x_i) = \frac{e^{x_i}}{\sum_j e^{x_j}} $$
Often we use $c = − \max x_i$. Why is that a good idea? (Tip: consider the range of values that numerator will have with $c = 0$ and $c = − \max x_i$)

$softmax(x_i + c) = \frac{e^{x_i}}{\sum_j e^{x_j}}   $ 

$= \frac{e^{x_i + c}}{\sum_j e^{x_j + c}}$

$= \frac{e^{x_i}e^c }{\sum_j e^{x_j} e^c} $

$= \frac{e^c}{e^c} \frac{e^{x_i}}{\sum_j e^{x_j}}$

$= \frac{e^{x_i}}{\sum_j e^{x_j}} $

$=  softmax(x_i)$

We use $c = -\max x_i$ to get a stable implementation of $softmax$.

This is because overflow can occur when computing the exponent if the values of $x$ are large, resulting in NaNs. 

By offsetting $x$ by the maximum value, we only have to calculate the exponent of values equal to or smaller than 0, which means we get the maximum precision of our float values.

(I referred to https://michielstock.github.io/posts/2021/2021-03-20-softmax/ for intuition on this answer.)

### Q3 (3 points)
Show that the functions represented by a multi-layer fully-connected neural networks without a non-linear activation function are linear functions of the input.

The output of a particular node in a particular layer is $ f(W*X + b)$ where $f$ is the activation function. Since $W*X + b$ is already linear in $X$, if $f$ is linear too, then this whole term is linear in $X$ as well.

If this is the case, then the output of each node is linear in its input $X$. 

Since the output of one layer (which we know to be linear) is the input to the subsequent layer (as $X$), then across all layers, the final output of the last layer is linear in the initial input to the first layer.

### Q4 (4 points) 
Given the sigmoid activation function $\sigma(x) = \frac{1}{1+e^{-x}}$ , derive the gradient of the sigmoid function and show that it can be written solely as a function of $\sigma(x)$.

$\frac{\partial }{\partial x} \sigma(x)$

$= \frac{\partial }{\partial x}  \frac{1}{1+e^{-x}} $

$= \frac{\partial }{\partial x}  (1+e^{-x})^{-1} $

$= -(1+e^{-x})^{-2}(-e^{-x})$

$= \frac{e^{-x}}{(1+e^{-x})^{2}}$

$= \frac{e^{-x}}{1+e^{-x}} \cdot \frac{1}{1+e^{-x}} $

$= \frac{(1 + e^{-x}) -1 }{1+e^{-x}} \cdot \frac{1}{1+e^{-x}} $

$=  (\frac{1 + e^{-x}}{1+e^{-x}} -  \frac{1}{1+e^{-x}}  ) \cdot \frac{1}{1+e^{-x}}  $

$=  (1 -  \frac{1}{1+e^{-x}}  ) \cdot \frac{1}{1+e^{-x}}$

$=  (1 -  \sigma(x)  ) \cdot \sigma(x) $

$=  (1 -  \sigma(x)  ) \cdot \sigma(x) $

### Q5 (12 points WriteUp)

Given $y = W x + b$ (or $y_j = \sum_{i=1}^d  x_{i} W_{ji} + b_j$), and the gradient of some loss $J$ with respect $y$, show how to get $\frac{\partial J}{\partial W}$, $\frac{\partial J}{\partial x}$ and $\frac{\partial J}{\partial b}$. Be sure to do the derivatives with scalars and re-form the matrix form afterwards. Here are some helpful notations.
$$ \frac{\partial J}{\partial y} = \delta \in \mathbb{R}^{k \times 1} \quad W \in \mathbb{R}^{k \times d} \quad x \in \mathbb{R}^{d \times 1} \quad b \in \mathbb{R}^{k \times 1}$$
    

$\frac{\partial J}{\partial W} = \frac{\partial J}{\partial y} \cdot \frac{\partial y}{\partial W} = \delta \cdot x$



$\frac{\partial J}{\partial x} = \frac{\partial J}{\partial y} \cdot \frac{\partial y}{\partial x} = \delta \cdot W$



$\frac{\partial J}{\partial b} = \frac{\partial J}{\partial y} \cdot \frac{\partial y}{\partial b} = \delta \cdot 1 = \delta $


### Q6 (15 points)

We will find the derivatives for Conv layers now. Since most Deep Learning frameworks such as Pytorch, Tensorflow use cross-correlation in their respective "convolution" functions ([Pytorch](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html#torch.nn.Conv2d) and [Tensorflow](https://www.tensorflow.org/api_docs/python/tf/nn/convolution)), we will continue this abuse of notation. So the operation performed with the Conv Layer weights will be cross-correlation.
    
The input, $x$ is of shape $M\times N$ with C channels. This will be *convolved* (actually cross-correlation) with $D$ number of $K\times K$ filters, each with a bias term. The stride is 1 and there will be no padding. We know the gradient of some loss $J$ with respect to the output $y$, which will have $D$ channels. Show how to get $\frac{\partial J}{\partial W}$, $\frac{\partial J}{\partial x}$ and $\frac{\partial J}{\partial b}$.

The dimensions and notation are as follows:
$$
    \frac{\partial J}{\partial y} = \delta \in \mathbb{R}^{D\times M_o \times N_o}
    \quad
    M_o = M-K+1
    \quad
    N_o = N-K+1
$$
$$
    x \in \mathbb{R}^{C\times M \times N}
    \quad
    W \in \mathbb{R}^{D\times C \times K \times K}
    \quad
    b \in \mathbb{R}^{D}
$$

$x_{c, i, j}:$ The element at the $i^{th}$ row, the $j^{th}$ column and the $c^{th}$ channel of the input

$y_{c, i, j}:$ The element at the $i^{th}$ row, the $j^{th}$ column and the $c^{th}$ channel of the output

$W_{d, c, i, j}:$ The element at the $i^{th}$ row, the $j^{th}$ column, the $c^{th}$ channel of the kernel of the $d^{th}$ filter

*For this question, you may compute the derivatives with scalars only. You don't need to re-form the matrix*
    

To get $\frac{\partial J}{\partial W}$, $\frac{\partial J}{\partial x}$ and $\frac{\partial J}{\partial b}$, we need to express $y$ as a function of $W$ and $x$. 

By the way, I'll be using the notation  $y_{d, i, j}$  instead of  $y_{c, i, j}$ since the first iterator ranges $1...D$ (not $1...C$).


$$y_{d, i, j} = b_d + \sum_{c=1}^{C} \sum_{k_i=1}^{K} \sum_{k_j=1}^{K} x_{c, i+k_i, j+k_j} \cdot W_{d, c, k_i, k_j}$$


Now for $\frac{\partial J}{\partial W}$. In scalar form, this is:
$$\frac{\partial J}{\partial W_{d, c, k_i, k_j}} = \frac{\partial J}{\partial y_{d,i,j}} \cdot \frac{\partial y_{d,i,j}}{\partial W_{d, c, k_i, k_j}} =  \sum_{c=1}^{C} \sum_{k_i=1}^{K} \sum_{k_j=1}^{K} \delta_{d,i,j} \cdot x_{c, i+k_i, j+k_j}

Now for  $\frac{\partial J}{\partial x}$. In scalar form, this is:
$$\frac{\partial J}{\partial x_{c, i+k_i, j+k_j}} = \frac{\partial J}{\partial y_{d,i,j}} \cdot \frac{\partial y_{d,i,j}}{\partial x_{c, i+k_i, j+k_j}} =  \sum_{c=1}^{C} \sum_{k_i=1}^{K} \sum_{k_j=1}^{K} \delta_{d,i,j} \cdot W_{d, c, k_i, k_j}

Now for  $\frac{\partial J}{\partial b}$. In scalar form, this is:
$$\frac{\partial J}{\partial b_d} = \frac{\partial J}{\partial y_{d,i,j}} \cdot \frac{\partial y_{d,i,j}}{\partial b_d} =   \delta_{d,i,j} 

### Q7 (4 points)

When the neural network applies the elementwise activation function (such as sigmoid), the gradient of the activation function scales the back-propagation update. This is directly from the chain rule, $\frac{d}{d x} f(g(x)) = f'(g(x)) g'(x)$.

#### Q7.1 (1 point)
Consider the sigmoid activation function for deep neural networks. Why might it lead to a "vanishing gradient" problem if it is used for many layers (consider plotting the $\sigma'(x)$ in Q1.4)?

Here is a graph of $\sigma(x)$ and its derivative:

<img align="center" src="sigmoid_graph.png" width="800">

Note that when $|x|$ is large, the derivative of the sigmoid gets squashed down close to 0.

When we do backprop, because of the chain rule, we multiply the derivatives of each layer down to the first layer. So if the derivatives are close to 0, and we are multiplying many of them together (in the case of a network with many layers), the gradient gets exponentially small.  

#### Q7.2 (1 point)
Often it is replaced with $\tanh(x) = \frac{1-e^{-2x}}{1+e^{-2x}}$. What are the output ranges of both $\tanh$ and sigmoid? Why might we prefer $\tanh$ ? 

The output range of $\tanh$ is [-1,1]. The output range of $\sigma(x)$ is [0,1].

We might prefer to use $\tanh$ since its output range is centered over 0, while $\sigma(x)$ is centered at 0.5. Convergence generally occurs faster if average of each input variable is close to 0. 

For instance. consider a large negative input to the activation functions. In $\sigma(x)$, this will map to a value close to 0. This means that updates of the weights will be slow in the backprop step. 

On the other hand with $\tanh$, a large negative input will still be mapped to a strongly-negative output. It is only input values that are near zero that will remain close to 0.  So, with $\tanh$ the network is less likely to get stuck during training.

#### Q7.3 (1 point)
Why does $\tanh(x)$ have less of a vanishing gradient problem? (plotting the derivatives helps! for reference: $\tanh'(x) = 1 - \tanh(x)^2$)

Here is a graph of the derivatives. $\tanh'(x)$ is green and $\sigma'(x)$ is black:

<img align="center" src="vanishing_grads.png" width="800">

Although both will have the vanishing gradient problem (since as $|x|$ gets very large, both derivatives approach 0), $\tanh'(x) $ will have slightly less of a vanishing gradient problem. This is because $\tanh'(x) $ ranges [0,1] while $\sigma'(x)$ ranges [0, 0.25], so with $\tanh(x)$ the updates to the weights are generally larger.

#### Q7.4 (1 point)
$\tanh$ is a scaled and shifted version of the sigmoid. Show how $\tanh(x)$ can be written in terms of $\sigma(x)$. (*Hint: consider how to make it have the same range*)

$\tanh(x) = \frac{1-e^{-2x}}{1+e^{-2x}} $

$ = \frac{(2-1) - e^{-2x}}{1+e^{-2x}} $  , substituting $2-1$ in for 1

$ = \frac{2-1-e^{-2x}}{1+e^{-2x}} $ 

$ = \frac{2}{1+e^{-2x}} - \frac{1+e^{-2x}}{1+e^{-2x}} $ 

$ = \frac{2}{1+e^{-2x}} - 1 $ 

$ = 2\cdot \frac{1}{1+e^{-2x}} - 1$ 

$ = 2\cdot \sigma(2x) - 1$ 